In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
cd "/content/drive/MyDrive/BTP_AMRITAMAM/"

/content/drive/MyDrive/BTP_AMRITAMAM


In [3]:
!pip install -q entropy==v0.1.0
!pip install pyeeg
!pip install eeglib

     |████████████████████████████████| 61kB 4.5MB/s 
     |████████████████████████████████| 204kB 11.0MB/s 
     |████████████████████████████████| 5.6MB 9.6MB/s 
     |████████████████████████████████| 256kB 39.3MB/s 
  Created wheel for websocket: filename=websocket-0.2.1-cp37-none-any.whl size=192135 sha256=257bc8e0477348d823cebc4c0b20b87541100fa59f9f9ec10177f50cfced7360
  Stored in directory: /root/.cache/pip/wheels/35/f7/5c/9e8243838269ea93f05295708519a6e183fa6b515d9ce3b636
Successfully built websocket
     |████████████████████████████████| 1.2MB 10.2MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for pyedflib: filename=pyEDFlib-0.1.20-cp37-cp37m-linux_x86_64.whl size=929377 sha256=936cf737d548caa55d5942481a7e85dc7987355867f757e4ea136532859ce802
  Stored in directory: /root/.cache/pip/wheels/0d/be/b5/574f8b99ad9ac2ecf78e0c5271040b5aff999febfe655b97a4
Successfully built pyedflib


In [9]:
import csv
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import plotly.graph_objects as go
import os  
import math
import statistics
from scipy.stats import kurtosis
import entropy as ent
#import pyeeg
import eeglib
import scipy as sp


def peakValue(X):
    n = X.shape[0]
    val = -100
    for i in range(0,n):
      tmp = abs(X.loc[i])
      if (tmp>val).bool():
        val = tmp

    return val

def rmsValue(X):
    n = X.shape[0]
    square = 0
    mean = 0.0
    root = 0.0
    for i in range(0,n):
      square += (X.loc[i]**2)
     
    mean = (square / (float)(n))
    root = math.sqrt(mean)
     
    print("root mean square value: ", root)
    print("---------------")
    return root

def absoluteMean(X):
    mean = 0;
    n = X.shape[0]
    for i in range(0,n):
        mean += abs(X.loc[i])
    
    print("Absolute mean: ", mean)
    print("---------------")
    return mean

def totalVariation(X):
  n = X.shape[0]
  val = 0
  for i in range(0,n-1):
    val += abs(X.loc[i+1] - X.loc[i])

  print("total variation: ", val)
  print("--------------")
  return val

def hurst(X):
    X = np.array(X)
    N = X.size
    T = np.arange(1, N + 1)
    Y = np.cumsum(X)
    Ave_T = Y / T

    S_T = np.zeros(N)
    R_T = np.zeros(N)

    for i in range(N):
        S_T[i] = np.std(X[:i + 1])
        X_T = Y - T * Ave_T[i]
        R_T[i] = np.ptp(X_T[:i + 1])

    R_S = R_T / S_T
    R_S = np.log(R_S)[1:]
    n = np.log(T)[1:]
    A = np.column_stack((n, np.ones(n.size)))
    [m, c] = np.linalg.lstsq(A, R_S, rcond = None)[0]
    H = m
    return H


def dfa(X, Ave=None, L=None):     # pyeeg official implementation
    """
    Finally, the relationship between F(n) and n is analyzed. A least square
    fitting is performed between log(F(n)) and log(n). The slope of the fitting
    line is the DFA value, denoted as Alpha. To white noise, Alpha should be
    0.5. Higher level of signal complexity is related to higher Alpha.
    Parameters
    ----------
    X:
        1-D Python list or numpy array
        a time series
    Ave:
        integer, optional
        The average value of the time series
    L:
        1-D Python list of integers
        A list of box size, integers in ascending order
    Returns
    -------
    Alpha:
        integer
        the result of DFA analysis, thus the slope of fitting line of log(F(n))
        vs. log(n). where n is the
    -----
    This value depends on the box sizes very much. When the input is a white
    noise, this value should be 0.5. But, some choices on box sizes can lead to
    the value lower or higher than 0.5, e.g. 0.38 or 0.58.
    Based on many test, I set the box sizes from 1/5 of    signal length to one
    (x-5)-th of the signal length, where x is the nearest power of 2 from the
    length of the signal, i.e., 1/16, 1/32, 1/64, 1/128, ...
    You may generate a list of box sizes and pass in such a list as a
    parameter.
    """

    X = pd.DataFrame(X).to_numpy()
    X = X.flatten()
    if Ave is None:
        Ave = np.mean(X)

    Y = np.cumsum(X)
    Y -= Ave

    if L is None:
        L = np.floor(len(X) * 1 / (
            2 ** np.array(list(range(4, int(np.log2(len(X))) - 4))))
        )

    F = np.zeros(len(L))  # F(n) of different given box length n

    for i in range(0, len(L)):
        n = int(L[i])                        # for each box length L[i]
        if n == 0:
            print("time series is too short while the box length is too big")
            print("abort")
            exit()
        for j in range(0, len(X), n):  # for each box
            if j + n < len(X):
                c = list(range(j, j + n))
                # coordinates of time in the box
                c = np.vstack([c, np.ones(n)]).T
                # the value of data in the box
                y = Y[j:j + n]
                # add residue in this box
                F[i] += np.linalg.lstsq(c, y)[1]
        F[i] /= ((len(X) / n) * n)
    F = np.sqrt(F)

    Alpha = np.linalg.lstsq(np.vstack(
        [np.log(L), np.ones(len(L))]
    ).T, np.log(F))[0][0]

    print("Detrended Fluctuation Analysis (dfa): ", Alpha)
    print("----------------")
    return Alpha


def hjorth(X, D=None):    # pyeeg official implementation
    """ Compute Hjorth mobility and complexity of a time series from either two
    cases below:
        1. X, the time series of type list (default)
        2. D, a first order differential sequence of X (if D is provided,
           recommended to speed up)
    In case 1, D is computed using Numpy's Difference function.
    Notes
    Returns
    -------
    As indicated in return line
    Hjorth mobility and complexity
    """
    #print("X: ", X)
    X = pd.DataFrame(X).to_numpy()
    X = X.flatten()
    #print("X: ", X)
    if D is None:
        D = np.diff(X)
        D = D.tolist()

    D.insert(0, X[0])  # pad the first difference
    D = np.array(D)

    n = len(X)

    M2 = float(sum(D ** 2)) / n
    TP = sum(np.array(X) ** 2)
    M4 = 0
    for i in range(1, len(D)):
        M4 += (D[i] - D[i - 1]) ** 2
    M4 = M4 / n

    return np.sqrt(M2 / TP), np.sqrt(
        float(M4) * TP / M2 / M2
    )  # Hjorth Mobility and Complexity


#from .embedded_sequence import embed_seq

def fisher_info(X, Tau, DE, W=None):    # pyeeg official implementation
    """Compute SVD Entropy from either two cases below:
    1. a time series X, with lag tau and embedding dimension dE (default)
    2. a list, W, of normalized singular values of a matrix (if W is provided,
    recommend to speed up.)
    Notes
    -------------
    To speed up, it is recommended to compute W before calling this function
    because W may also be used by other functions whereas computing it here
    again will slow down.
    """

    if W is None:
        Y = embed_seq(X, Tau, DE)
        W = numpy.linalg.svd(Y, compute_uv=0)
        W /= sum(W)  # normalize singular values

    return (-1 * sum(W * numpy.log(W)))

s = "A01T_1";   #read file name
readFile = "Segmentation/A01/Training/" + s  + "/1.csv"  #read file location
saveDirectory = "Graphs/A01/Training/"  #save location
training = pd.read_csv(readFile)
#makeFolder(s,saveDirectory)
print(training.shape[1])
n = training.shape[1]

for node_no in range (0,1):
    labelling = [0] * 22
    i = 0
    # Get Kth Column of Matrix
    x = training.iloc [ : , node_no :node_no+1 ]
    print(x.shape[1])
    print(x.shape[0])
    labelling[0] = kurtosis(x, fisher = True)  #fisher: If True, Fisher’s definition is used (normal ==> 0.0). If False, Pearson’s definition is used (normal ==> 3.0).
    print("Kurtosis: ", labelling[0], "\n--------------")
    labelling[1] = peakValue(x)
    labelling[2] = rmsValue(x)
    labelling[3] = (labelling[1]/labelling[2])**2   # Peak-to-Average Power Ratio
    labelling[4] = (labelling[2]/absoluteMean(x))   # Form factor
    labelling[5] = totalVariation(x)
    labelling[6] = hurst(x)
    print("Hurst exponent: ", labelling[6], "\n--------------")
    #labelling[7] = {ent.detrended_fluctuation(x)}
    labelling[7] = dfa(x)
    labelling[8], labelling[9] = hjorth(x)
    print("Hjorth mobility, complexity: ", labelling[8], " ", labelling[9])
    print("-------------------")
    #labelling[10] = fisher_info(x)
    


22
1
1000
Kurtosis:  [0.23305583] 
--------------
root mean square value:  3.0158556681044293
---------------
Absolute mean:  1    2366.617917
Name: 0, dtype: float64
---------------
total variation:  1    986.481742
dtype: float64
--------------
Hurst exponent:  0.24517773269365123 
--------------
Detrended Fluctuation Analysis (dfa):  0.4990422288500693
----------------
Hjorth mobility, complexity:  0.013118815829522075   38.853097400206984
-------------------


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:76: RuntimeWarning:

invalid value encountered in true_divide

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:148: FutureWarning:

`rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:154: FutureWarning:

`rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.

